<font size="3">KINGDOM OF SAUDI ARABIA<br>
Ministry of Higher Education<br>                                  
Al-Imam Mohammad University<br>
College of Computer & Information Sciences<br><br>
    
 
**Deep Learning (CS464), Winter 22-23 - Second Semester 1444**
<br>**Instructor:
Dr. Haifa Alkasem**
<br><br>
**Prepared by:**<br>
Raghad Albosais (440020209)<br>

**Section: 371**

This is seperate notebook from the original notebook. Since I faced a problem regarding the memory usage, during the run time of hyperparameters tunning, it full up until at certain points, it is crashe. This is due to the natural work of GridSearch, it is brute-force approaches to finding the right
hyperparameter configurations, which is an expensive and time-consuming process. (if we have two
parameters specified, each of with 2 values, and the folds is 3. It ends up with 2x2=4 candidate
models, each model with 3 folds, totalling of 4x3 = 12 fit models.)
Therefore, what I did to overcome the limited resources I have is to split the hyperparameter tunning
into another notebook with only necessary cells to do it (without data visualization and etc.)

In [ ]:
# Importing all the libraries 

#  enables the drawing of matplotlib figures in the IPython environment.
%matplotlib inline

# to access the path
import os

# libraries for visualization
import seaborn as sns
import matplotlib.pyplot as plt

# tensorflow to build and process DL model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD

# library to work with dataframes
import pandas as pd
# library to work on array and matrcies
import numpy as np
# library to work on basic operation of ML project
from sklearn.metrics import accuracy_score, classification_report

# libraries to download images
import cv2
from PIL import Image

# implement scikit-learn classifier API for Keras
from keras.wrappers.scikit_learn import KerasClassifier

# cross validation methods
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# hyperparameters tunning method
from sklearn.model_selection import GridSearchCV

# to ensure reproducibility of the result in each run
tf.random.set_seed(1234)

# remove messages
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

In [ ]:
# check if CUDA is available
import warnings
warnings.filterwarnings('ignore')
train_on_gpu = tf.test.is_gpu_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


# Loading the Dataset

In [ ]:
# define train, test and valid data directories
data_dir = 'afterSplit_chest_xray'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
val_dir = os.path.join(data_dir, 'val')

In [ ]:
# download the images as filenames with their labels
labels = ['NORMAL', 'PNEUMONIA']
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            if img[-4:] == 'jpeg':
                data.append((os.path.join(path, img), class_num))

    return data
    
train = get_data(train_dir)
test = get_data(test_dir)
val = get_data(val_dir)

In [ ]:
# specify the image classes
classes = os.listdir(train_dir)
calsses_name = dict()
for name in classes:
    if name == 'NORMAL':
        calsses_name[name] = 0
    else:
        calsses_name[name] = 1 
print('Num of classes: ', len(classes))
print('Classes names: ', calsses_name)

Num of classes:  2
Classes names:  {'PNEUMONIA': 1, 'NORMAL': 0}


In [ ]:
# download the dataset as (image, label) pairs

# list of class names
labels = ['PNEUMONIA', 'NORMAL']

# hyperparameter > the size of image
img_size = 224
# hyperparameter > the channel of image
channel = cv2.IMREAD_COLOR

# function take the directory and return the data in pairs (image, label)
def get_data(data_dir):
    data = []
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)

        for img in os.listdir(path):

            if img[-4:] == 'jpeg':
                try:
                    img_arr = cv2.imread(os.path.join(path, img), channel)

                    # ---- basic preprocessing ---- #
                    # 1. reshaping images to preferred size
                    resized_arr = cv2.resize(img_arr, (img_size, img_size))
                    # 2. change to float datatype
                    img = resized_arr.astype('float32') 
                    # 3. normalize the pixels value to lie between 0 to 1
                    img = img / 255.0 

                    data.append([img, class_num])

                except Exception as e:
                    print(e)

    return np.array(data)

train_data = get_data(train_dir)
test_data = get_data(test_dir)
valid_data = get_data(val_dir)

In [ ]:
# further split each pair to be x for data and y for label
x_train = []
y_train = []

x_val = []
y_val = []

x_test = []
y_test = []

for feature, label in train_data:
    x_train.append(feature)
    y_train.append(label)

for feature, label in test_data:
    x_test.append(feature)
    y_test.append(label)
    
for feature, label in valid_data:
    x_val.append(feature)
    y_val.append(label)

In [ ]:
# convert the lists of x and y into numpy arrays
# so that to make it suitable as input for tensorflow methods (fit, evaluate, predict)
x_train = np.array(x_train)
x_val = np.array(x_val)
x_test = np.array(x_test)

y_train = np.array(y_train).reshape(len(y_train), 1)
y_val = np.array(y_val).reshape(len(y_val),1)
y_test = np.array(y_test).reshape(len(y_test), 1)

# Hyper-parameters tuning with k-folds

In [ ]:
# combine the train and validation into one set
X_train = np.concatenate((x_train, x_val))
print(X_train.shape)

Y_train = np.concatenate((y_train, y_val))
print(Y_train.shape)

K-Folds cross-validator as hyperparameters tunning and evaluation for different hyperparameters values using  GridSearch and KerasClassifier.

In [ ]:
# model arichitecture and configuration
def create_model(optimizer='adam'):
    # create the model using sequential method, where the order of layers will be as added
    model = Sequential()

    # first conv bolck
    # input 224x224x1, output 112x112x16
    model.add(Conv2D(16, (3, 3), padding='same', input_shape=(img_size, img_size, 3)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # second conv block
    # input 112x112x16, output 56x56x32
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # third conv block
    # input 56x56x32, output 28x28x64
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # forth conv block
    # input 28x28x64, output 14x14x128
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # flatt the last conv block ito 1D vector
    # input 14x14x128, output 25088
    model.add(Flatten())

    # first FC layer
    # input 25088, output 512
    model.add(Dense(512))
    model.add(Activation("relu"))

    # second FC layer
    # input 512, output 128
    model.add(Dense(128))
    model.add(Activation("relu"))

    # third FC layer
    # input 128, output 64
    model.add(Dense(64))
    model.add(Activation("relu"))

    # forth FC layer
    # input 64, output 1
    model.add(Dense(1))
    model.add(Activation("sigmoid"))

    loss = 'binary_crossentropy'
    metrics = ['accuracy',
    tf.keras.metrics.Precision(name='precision'), 
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.TruePositives(name='TP'),
    tf.keras.metrics.TrueNegatives(name='TN'),
    tf.keras.metrics.FalsePositives(name='FP'),
    tf.keras.metrics.FalseNegatives(name='FN')]

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)
    
    return model

In [ ]:
# using k fold as hyperparamer tunning 

# define cross-validation method
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

# different variation of k fold cross validator
skf = StratifiedKFold(n_splits=3, shuffle= True, random_state= 17)
kf = KFold(n_splits=3, shuffle= True, random_state= 17)

# define Model wihtout specifing hyperparamers value
model3 = KerasClassifier(build_fn=create_model,
                        verbose=1,
                        shuffle=True)

# define Hyper-parameter and its list values
optimizer =  ['SGD', 'adam'] 
batch_size = [16, 32]
epochs = [5, 10]

param_grid = dict(optimizer=optimizer,batch_size=batch_size, epochs=epochs)

# define the evaluate the performance of the cross-validated model on the test set
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
scoring = make_scorer(accuracy_score)

# define GridSearchCV and fit it in the model
grid = GridSearchCV(estimator=model3, param_grid=param_grid, refit = True, scoring = scoring, cv=skf)
grid_model = grid.fit(x_train, y_train)

Epoch 1/5
285/285 [==============================] - 5s 13ms/step - loss: 0.2598 - accuracy: 0.8938 - precision: 0.8815 - recall: 0.9101 - TP: 2075.0000 - TN: 1999.0000 - FP: 279.0000 - FN: 205.0000
Epoch 2/5
285/285 [==============================] - 3s 12ms/step - loss: 0.1320 - accuracy: 0.9495 - precision: 0.9457 - recall: 0.9539 - TP: 2175.0000 - TN: 2153.0000 - FP: 125.0000 - FN: 105.0000
Epoch 3/5
285/285 [==============================] - 3s 12ms/step - loss: 0.0913 - accuracy: 0.9680 - precision: 0.9680 - recall: 0.9680 - TP: 2207.0000 - TN: 2205.0000 - FP: 73.0000 - FN: 73.0000
Epoch 4/5
285/285 [==============================] - 3s 12ms/step - loss: 0.0717 - accuracy: 0.9756 - precision: 0.9754 - recall: 0.9759 - TP: 2225.0000 - TN: 2222.0000 - FP: 56.0000 - FN: 55.0000
Epoch 5/5
72/72 [==============================] - 1s 8ms/step
Epoch 1/5
285/285 [==============================] - 5s 12ms/step - loss: 0.2627 - accuracy: 0.8989 - precision: 0.8925 - recall: 0.9070 - TP: 20

In [ ]:
# the best Model
print("Best: %f using %s" % (grid_model.best_score_, grid_model.best_params_))

# print all experiment results
import pandas as pd
results = pd.DataFrame(grid_model.cv_results_)
results

Best: 0.963876 using {'batch_size': 32, 'epochs': 10, 'optimizer': 'SGD'}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_epochs,param_optimizer,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,21.022145,0.057674,1.261626,0.025074,16,5,SGD,"{'batch_size': 16, 'epochs': 5, 'optimizer': '...",0.972368,0.953927,0.960070,0.962122,0.007667,2
1,22.139889,0.123384,1.244888,0.016963,16,5,adam,"{'batch_size': 16, 'epochs': 5, 'optimizer': '...",0.561842,0.912242,0.893813,0.789299,0.161012,8
2,38.600697,0.246057,1.231761,0.010854,16,10,SGD,"{'batch_size': 16, 'epochs': 10, 'optimizer': ...",0.948684,0.966213,0.954366,0.956421,0.007302,4
3,40.460346,0.083299,1.253082,0.022977,16,10,adam,"{'batch_size': 16, 'epochs': 10, 'optimizer': ...",0.965351,0.964458,0.501097,0.810302,0.218641,7
4,17.402385,0.032225,1.240948,0.010374,32,5,SGD,"{'batch_size': 32, 'epochs': 5, 'optimizer': '...",0.970175,0.953050,0.907416,0.943547,0.026488,5
5,18.116621,0.201200,1.248540,0.027881,32,5,adam,"{'batch_size': 32, 'epochs': 5, 'optimizer': '...",0.946491,0.955244,0.874068,0.925267,0.036380,6
6,31.528409,0.222297,1.251847,0.008125,32,10,SGD,"{'batch_size': 32, 'epochs': 10, 'optimizer': ...",0.978947,0.968407,0.944274,0.963876,0.014513,1
7,32.457918,0.017717,1.249578,0.015695,32,10,adam,"{'batch_size': 32, 'epochs': 10, 'optimizer': ...",0.957018,0.963581,0.956121,0.958906,0.003325,3


In [ ]:
# print the best value of precesion score
print('Best Score: %s' % grid_model.best_score_)

# print best parameter after tuning
print("\nBest parameters set:")
print(grid_model.best_params_)
     

Best Score: 0.9638761229532805

Best parameters set:
{'batch_size': 32, 'epochs': 10, 'optimizer': 'SGD'}


In [ ]:
# score on the refit data
scores = grid_model.score(x_test, y_test)
print(scores)

# predict on the refit data
predictions = grid_model.predict(x_test)
print(predictions)

19/19 [==============================] - 0s 7ms/step
0.959114139693356
19/19 [==============================] - 0s 7ms/step
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 